In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import numpy as np
import os
from tqdm import tqdm
import pickle
import cv2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, select_largest=False, keep_all=True,
    device=device
)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def get_embeddings(image):
    image = cv2.imread(image)
    x_aligned = mtcnn(image)
    if x_aligned is not None:
        return resnet(x_aligned.to(device)).detach().cpu().numpy()
    return np.array([])

In [3]:
CHANNEL_ID = # your gallery channel id

embedding_to_file = {}
for path, dirs, files in os.walk('data/'):
    for file in tqdm(sorted(files, reverse=True)):
        embeddings = get_embeddings(path + file)
        for embedding in embeddings:
            embedding_to_file[tuple(embedding)] = (CHANNEL_ID, int(file[:-4]))

100%|██████████| 800/800 [02:00<00:00,  6.64it/s]


In [4]:
with open('embeddings_event.idx', 'wb') as f:
    pickle.dump(embedding_to_file, f)